In [1]:
import pandas as pd
pd.set_option('display.max_rows', 200)
import numpy as np
import xlrd
import math

import geopy
from geopy.geocoders import Nominatim
import requests
import random
import itertools
import folium
import json
import mpl_toolkits
from folium import plugins
import matplotlib.pylab as plt
from matplotlib.ticker import NullFormatter
import matplotlib.ticker as ticker
import matplotlib as mpl
from mpl_toolkits.basemap import Basemap
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.image as mpimg
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

import urllib
from bs4 import BeautifulSoup
import ssl

from IPython.display import Image
from IPython.core.display import HTML
from pandas.io.json import json_normalize

import pylab as pl
from pylab import rcParams
import pydotplus
import numba
import pickle
import scipy as sp
from scipy import ndimage
from scipy.cluster import hierarchy
from scipy.cluster.hierarchy import fcluster
from scipy.spatial import distance_matrix
from scipy.optimize import curve_fit
from scipy import optimize as opt
import seaborn as sns

import statsmodels as sm
import wget


import ipywidgets as widgets
from ipywidgets import interact
from ipywidgets import interactive
from ipywidgets import fixed
from ipywidgets import interact_manual


import sklearn as sk
import sklearn.utils
from sklearn import manifold
from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from sklearn.datasets.samples_generator import make_blobs
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from sklearn.metrics import r2_score
from sklearn.metrics import log_loss
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.externals.six import StringIO

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


## PART 1

In [16]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = urllib.request.urlopen(url, context=ctx).read()
soup = BeautifulSoup(html, 'html.parser')
data = soup.get_text()
data = data[4349:13695]
data = data.rstrip().split('\n')
postcode = []
borough = []
neighborhood = []
q = 4
i = 0
for line in data :
    if i % 5 == 0 and i > 4:
        postcode.append(line)
    elif i % 5 == 1 and i > 4:
        borough.append(line)
    elif i % 5 == 2 and i > 4:
        if line == 'Not assigned' :
            line = q
        neighborhood.append(line)
    i = i + 1
    q = line
df = pd.DataFrame(list(zip(postcode, borough , neighborhood)), 
               columns =['Postcode', 'Borough' , 'Neighborhood'])
df.drop(df.loc[df['Borough']== 'Not assigned'].index, inplace=True)
df = df.groupby(['Postcode','Borough'], 
                  as_index=False)['Neighborhood'].agg(lambda x: list(x))
df['Neighborhood'] = df.Neighborhood.apply(', '.join)
df1 = pd.read_csv('gc.csv')
df1.rename({'Postal Code': 'Postcode'}, axis=1, inplace=True)
df = df.merge(df1, how='right', on='Postcode')
df.shape
df.columns

Index(['Postcode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'], dtype='object')

In [18]:
latitude = 43.761539
longitude = -79.411079
map_Toronto = folium.Map(location = [latitude , longitude] , zoom_start = 10)

for lat , lng , Borough , Neighborhood , Postcode in zip(df['Latitude'] , df['Longitude'] , df['Borough'] , df['Neighborhood'] , df['Postcode']) :
    label = '{} , {} , {}'.format(Postcode , Borough , Neighborhood)
    label = folium.Popup(label , parse_html = True)
    folium.CircleMarker(
        [lat , lng] ,
        radius = 5 ,
        popup = label ,
        color = 'blue' ,
        fill = True ,
        fill_color = '#3186cc' ,
        fill_opacity = 0.7 ,
        parse_html = False).add_to(map_Toronto)
    

In [19]:
map_Toronto